<font size=6>
    <b>Model_Training_with_TensorArray.ipynb:</b>
    <p>Use Text Extensions for Pandas TensorArray to integrate model training and prediction with Pandas.</p>
</font>

# Introduction

This notebook shows how to use the open source library [Text Extensions for Pandas](https://github.com/CODAIT/text-extensions-for-pandas) to seamlessly integrate model training and prediction with [Pandas](https://pandas.pydata.org/) DataFrames for added support of columns with N-dimensional tensors using the Pandas extension array called `TensorArray`.

## Model Training Example

To demonstrate the TensorArray functionality from Text Extensions for Pandas, we will use the CoNLL-2003 corpus used from https://www.aclweb.org/anthology/W03-0419/ to ***
This notebook will build on the analysis on the CoNLL-2003 corpus done in [Analyze_Model_Outputs](./Analyze_Model_Outputs.ipynb), and train a new model using state-of-the-art natural language understanding with BERT tokenization and embeddings.

TODO
* explain BERT
* mention huggingface transformers and how it is integrated, https://huggingface.co/transformers/index.html

1. Retokenize the entire corpus using a "BERT-compatible" tokenizer, and map the token/entity labels from the original corpus on to the new tokenization.
1. Generate BERT embeddings for every token in the entire corpus in one pass, and store those embeddings in a DataFrame column (of type TensorType) alongside the tokens and labels.
1. Persist the DataFrame with computed BERT embeddings to disk as a checkpoint.
1. Use the embeddings to quickly train a model that will classify token class.
1. Make predictions of the token classes on the test set, adding predicted class probabilities as a TensorArray to the DataFrame.
1. Drill down into the DataFrame with some analysis on the predictions.


## Environment Setup

This notebook requires a Python 3.7 or later environment with NumPy, Pandas, scikit-learn and Huggingface `transformers`. 

The notebook also requires the  `text_extensions_for_pandas` library. You can satisfy this dependency in two ways:

* Run `pip install text_extensions_for_pandas` before running this notebook. This command adds the library to your Python environment.
* Run this notebook out of your local copy of the Text Extensions for Pandas project's [source tree](https://github.com/CODAIT/text-extensions-for-pandas). In this case, the notebook will use the version of Text Extensions for Pandas in your local source tree **if the package is not installed in your Python environment**.

In [39]:
import os
import sys
from typing import *
import numpy as np
import pandas as pd
import sklearn.pipeline
import sklearn.linear_model
import transformers

# And of course we need the text_extensions_for_pandas library itself.
try:
    import text_extensions_for_pandas as tp
except ModuleNotFoundError as e:
    # If we're running from within the project source tree and the parent Python
    # environment doesn't have the text_extensions_for_pandas package, use the
    # version in the local source tree.
    if not os.getcwd().endswith("notebooks"):
        raise e
    if ".." not in sys.path:
        sys.path.insert(0, "..")
    import text_extensions_for_pandas as tp

# Example usage using BERT embeddings for Model Training on CoNLL-2003

Now let's put the TensorArray to use with an example. Here we use the CoNLL-2003 corpus and retokenize using a BERT tokenizer. Next, the BERT tokens will be used to compute BERT embeddings that will be stored in a TensorArray and added to the DataFrame. Finally, we can use the BERT embeddings to train a model and make predictions on the token class, which will also be added to the DataFrame as a TensorArray

In [3]:
# Download and cache the data set.
# NOTE: This data set is licensed for research use only. Be sure to adhere
#  to the terms of the license when using this data set!
data_set_info = tp.io.conll.maybe_download_conll_data("outputs")
data_set_info

{'train': 'outputs/eng.train',
 'dev': 'outputs/eng.testa',
 'test': 'outputs/eng.testb'}

# Show how to retokenize with a BERT tokenizer.

Retokenize the corpus using a "BERT-compatible" tokenizer, and map the token/entity labels from the original corpus on to the new tokenization.

### Prepare the corpus for tokenization

In [4]:
# Read in the corpus in its original tokenization
corpus_raw = {}
for fold_name, file_name in data_set_info.items():
    df_list = tp.io.conll.conll_2003_to_dataframes(file_name, 
                                                   ["pos", "phrase", "ent"],
                                                   [False, True, True])
    corpus_raw[fold_name] = [
        df.drop(columns=["pos", "phrase_iob", "phrase_type"])
        for df in df_list
    ]

test_raw = corpus_raw["test"]

# Pick out the dataframe for a single example document.
example_df = test_raw[5]
example_df

,span,ent_iob,ent_type,sentence,line_num
0,"[0, 10): '-DOCSTART-'",O,None,"[0, 10): '-DOCSTART-'",1469
1,"[11, 18): 'CRICKET'",O,None,"[11, 62): 'CRICKET- PAKISTAN V NEW ZEALAND ONE...",1471
2,"[18, 19): '-'",O,None,"[11, 62): 'CRICKET- PAKISTAN V NEW ZEALAND ONE...",1472
3,"[20, 28): 'PAKISTAN'",B,LOC,"[11, 62): 'CRICKET- PAKISTAN V NEW ZEALAND ONE...",1473
4,"[29, 30): 'V'",O,None,"[11, 62): 'CRICKET- PAKISTAN V NEW ZEALAND ONE...",1474
...,...,...,...,...,...
350,"[1620, 1621): '8'",O,None,"[1590, 1634): 'Third one-day match: December 8...",1865
351,"[1621, 1622): ','",O,None,"[1590, 1634): 'Third one-day match: December 8...",1866
352,"[1623, 1625): 'in'",O,None,"[1590, 1634): 'Third one-day match: December 8...",1867
353,"[1626, 1633): 'Karachi'",B,LOC,"[1590, 1634): 'Third one-day match: December 8...",1868


## Convert IOB-Tagged Data to Lists of Entity Mentions

**TODO** reword for this nb

The data we've looked at so far has been in [IOB2 format](https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging)). 
Each row of our DataFrame represents a token, and each token is tagged with an entity type (`ent_type`) and an IOB tag (`ent_iob`). The first token of each named entity mention is tagged `B`, while subsequent tokens are tagged `I`. Tokens that aren't part of any named entity are tagged `O`.

IOB2 format is a convenient way to represent a corpus, but it is a less useful representation for analyzing the result quality of named entity recognition models. Most tokens in a typical NER corpus will be tagged `O`, any measure of error rate in terms of tokens will over-emphasizing the tokens that are part of entities. Token-level error rate implicitly assigns higher weight to named entity mentions that consist of multiple tokens, further unbalancing error metrics. And most crucially, a naive comparison of IOB tags can result in marking an incorrect answer as correct. Consider a case where the correct sequence of labels is `B, B, I` but the model has output `B, I, I`; in this case, last two tokens of model output are both incorrect (the model has assigned them to the same entity as the first token), but a naive token-level comparison will consider the last token to be correct.

The CoNLL 2003 competition used the number of errors in extracting *entire* entity mentions to measure the result quality of the entries. We will use the same metric in this notebook. To compute entity-level errors, we convert the IOB-tagged tokens into pairs of `<entity span,  entity type>`. 
Text Extensions for Pandas includes a function `iob_to_spans()` that will handle this conversion for you.

In the next cell, we use `iob_to_spans()` to convert both the corpus and our example model's output to DataFrames of entity span and type information. Then we display the `<entity span,  entity type>` pairs for the "bender" team's output on our example document.

In [7]:
spans_df = tp.io.conll.iob_to_spans(example_df)
spans_df

,span,ent_type
0,"[20, 28): 'PAKISTAN'",LOC
1,"[31, 42): 'NEW ZEALAND'",LOC
2,"[80, 83): 'GMT'",MISC
3,"[85, 92): 'SIALKOT'",LOC
4,"[94, 102): 'Pakistan'",LOC
...,...,...
69,"[1488, 1501): 'Shahid Afridi'",PER
70,"[1512, 1523): 'Salim Malik'",PER
71,"[1535, 1545): 'Ijaz Ahmad'",PER
72,"[1565, 1573): 'Pakistan'",LOC


### Initialize our BERT Tokenizer and Model

TODO talk about BERT

In [8]:
# Huggingface transformers BERT Configuration
bert_model_name = "dslim/bert-base-NER"

tokenizer = transformers.BertTokenizerFast.from_pretrained(bert_model_name, 
                                                           add_special_tokens=True)
bert = transformers.BertModel.from_pretrained(bert_model_name)

In [11]:
# Retokenize the document's text with the BERT tokenizer
bert_toks_df = tp.io.bert.make_bert_tokens(example_df["span"].values[0].target_text, tokenizer)
bert_toks_df

,token_id,span,input_id,token_type_id,attention_mask,special_tokens_mask
0,0,"[0, 0): ''",101,0,1,True
1,1,"[0, 1): '-'",118,0,1,False
2,2,"[1, 2): 'D'",141,0,1,False
3,3,"[2, 4): 'OC'",9244,0,1,False
4,4,"[4, 6): 'ST'",9272,0,1,False
...,...,...,...,...,...,...
684,684,"[1621, 1622): ','",117,0,1,False
685,685,"[1623, 1625): 'in'",1107,0,1,False
686,686,"[1626, 1633): 'Karachi'",16237,0,1,False
687,687,"[1633, 1634): '.'",119,0,1,False


In [12]:
# BERT tokenization includes special zero-length tokens.
bert_toks_df[bert_toks_df["special_tokens_mask"]]

,token_id,span,input_id,token_type_id,attention_mask,special_tokens_mask
0,0,"[0, 0): ''",101,0,1,True
688,688,"[0, 0): ''",102,0,1,True


In [16]:
# Align the BERT tokens with the original tokenization
bert_spans = tp.TokenSpanArray.align_to_tokens(bert_toks_df["span"],
                                               spans_df["span"])
pd.DataFrame({
    "original_span": spans_df["span"],
    "bert_spans": bert_spans,
    "ent_type": spans_df["ent_type"]
})

,original_span,bert_spans,ent_type
0,"[20, 28): 'PAKISTAN'","[20, 28): 'PAKISTAN'",LOC
1,"[31, 42): 'NEW ZEALAND'","[31, 42): 'NEW ZEALAND'",LOC
2,"[80, 83): 'GMT'","[80, 83): 'GMT'",MISC
3,"[85, 92): 'SIALKOT'","[85, 92): 'SIALKOT'",LOC
4,"[94, 102): 'Pakistan'","[94, 102): 'Pakistan'",LOC
...,...,...,...
69,"[1488, 1501): 'Shahid Afridi'","[1488, 1501): 'Shahid Afridi'",PER
70,"[1512, 1523): 'Salim Malik'","[1512, 1523): 'Salim Malik'",PER
71,"[1535, 1545): 'Ijaz Ahmad'","[1535, 1545): 'Ijaz Ahmad'",PER
72,"[1565, 1573): 'Pakistan'","[1565, 1573): 'Pakistan'",LOC


In [19]:
# Generate IOB2 tags and entity labels that align with the BERT tokens.
# See https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging)
bert_toks_df[["ent_iob", "ent_type"]] = tp.io.conll.spans_to_iob(bert_spans, 
                                                        spans_df["ent_type"])
bert_toks_df

,token_id,span,input_id,token_type_id,attention_mask,special_tokens_mask,ent_iob,ent_type
0,0,"[0, 0): ''",101,0,1,True,O,<NA>
1,1,"[0, 1): '-'",118,0,1,False,O,<NA>
2,2,"[1, 2): 'D'",141,0,1,False,O,<NA>
3,3,"[2, 4): 'OC'",9244,0,1,False,O,<NA>
4,4,"[4, 6): 'ST'",9272,0,1,False,O,<NA>
...,...,...,...,...,...,...,...,...
684,684,"[1621, 1622): ','",117,0,1,False,O,<NA>
685,685,"[1623, 1625): 'in'",1107,0,1,False,O,<NA>
686,686,"[1626, 1633): 'Karachi'",16237,0,1,False,B,LOC
687,687,"[1633, 1634): '.'",119,0,1,False,O,<NA>


In [24]:
# Create a Pandas categorical type for consistent encoding of categories
# across all documents.
ENTITY_TYPES = ["LOC", "MISC", "ORG", "PER"]
token_class_dtype, int_to_label, label_to_int = tp.io.conll.make_iob_tag_categories(ENTITY_TYPES)
token_class_dtype

CategoricalDtype(categories=['O', 'B-LOC', 'B-MISC', 'B-ORG', 'B-PER', 'I-LOC', 'I-MISC',
                  'I-ORG', 'I-PER'],
                 ordered=False)

In [25]:
# The traditional way to transform NER to token classification is to 
# treat each combination of {I,O,B} X {entity type} as a different
# class. Generate class labels in that format.
classes_df = tp.io.conll.add_token_classes(bert_toks_df, token_class_dtype)
classes_df

,token_id,span,input_id,token_type_id,attention_mask,special_tokens_mask,ent_iob,ent_type,token_class,token_class_id
0,0,"[0, 0): ''",101,0,1,True,O,<NA>,O,0
1,1,"[0, 1): '-'",118,0,1,False,O,<NA>,O,0
2,2,"[1, 2): 'D'",141,0,1,False,O,<NA>,O,0
3,3,"[2, 4): 'OC'",9244,0,1,False,O,<NA>,O,0
4,4,"[4, 6): 'ST'",9272,0,1,False,O,<NA>,O,0
...,...,...,...,...,...,...,...,...,...,...
684,684,"[1621, 1622): ','",117,0,1,False,O,<NA>,O,0
685,685,"[1623, 1625): 'in'",1107,0,1,False,O,<NA>,O,0
686,686,"[1626, 1633): 'Karachi'",16237,0,1,False,B,LOC,B-LOC,1
687,687,"[1633, 1634): '.'",119,0,1,False,O,<NA>,O,0


## Show how to compute BERT embeddings

We are going to use the BERT embeddings as the feature vector to train our model. First, we will show how they are computed 

In [27]:
# Add BERT embeddings to our example dataframe.
embeddings_df = tp.io.bert.add_embeddings(classes_df, bert)
embeddings_df

,token_id,span,input_id,token_type_id,attention_mask,special_tokens_mask,ent_iob,ent_type,token_class,token_class_id,embedding
0,0,"[0, 0): ''",101,0,1,True,O,<NA>,O,0,[-8.30711797e-02 -3.59590203e-01 1.01506817e+0...
1,1,"[0, 1): '-'",118,0,1,False,O,<NA>,O,0,[-2.28625670e-01 -4.93136197e-01 1.28423226e+0...
2,2,"[1, 2): 'D'",141,0,1,False,O,<NA>,O,0,[ 2.84805615e-02 -1.78742811e-01 1.54320943e+0...
3,3,"[2, 4): 'OC'",9244,0,1,False,O,<NA>,O,0,[-0.46517587 -0.2983605 1.0737675 -0.0316486...
4,4,"[4, 6): 'ST'",9272,0,1,False,O,<NA>,O,0,[-1.07308812e-01 -3.37210178e-01 1.22697961e+0...
...,...,...,...,...,...,...,...,...,...,...,...
684,684,"[1621, 1622): ','",117,0,1,False,O,<NA>,O,0,[-1.28065705e-01 -2.32442445e-03 6.78131640e-0...
685,685,"[1623, 1625): 'in'",1107,0,1,False,O,<NA>,O,0,[ 0.30534053 -0.526257 0.8281703 -0.2741491...
686,686,"[1626, 1633): 'Karachi'",16237,0,1,False,B,LOC,B-LOC,1,[-0.04873929 -0.33797333 -0.05835138 0.7557763...
687,687,"[1633, 1634): '.'",119,0,1,False,O,<NA>,O,0,[-5.28932782e-03 -2.97430724e-01 7.16174304e-0...


In [28]:
# The `embedding` column is an extension type `TensorDtype` that holds a `TensorArray` provided by Text Extensions for Pandas.
embeddings_df["embedding"].dtype

In [29]:
# A `TensorArray` can be constructed with a NumPy array of arbitrary dimensions and then used with standard Pandas functionality and added to a DataFrame.
# Zero-copy conversion to NumPy can be done by first unwrapping the `TensorArray` with `.array` and calling `to_numpy()`
embeddings_arr = embeddings_df["embedding"].array.to_numpy()
embeddings_arr.shape

(689, 768)

## Generate BERT tokens and BERT embeddings for the entire corpus

In [31]:
# Convienence function to combine the previous few cells' operations in a single call.
tp.io.bert.conll_to_bert(example_df, tokenizer, bert, token_class_dtype)

,token_id,span,input_id,token_type_id,attention_mask,special_tokens_mask,ent_iob,ent_type,token_class,token_class_id,embedding
0,0,"[0, 0): ''",101,0,1,True,O,<NA>,O,0,[-8.30711797e-02 -3.59590203e-01 1.01506817e+0...
1,1,"[0, 1): '-'",118,0,1,False,O,<NA>,O,0,[-2.28625670e-01 -4.93136197e-01 1.28423226e+0...
2,2,"[1, 2): 'D'",141,0,1,False,O,<NA>,O,0,[ 2.84805615e-02 -1.78742811e-01 1.54320943e+0...
3,3,"[2, 4): 'OC'",9244,0,1,False,O,<NA>,O,0,[-0.46517587 -0.2983605 1.0737675 -0.0316486...
4,4,"[4, 6): 'ST'",9272,0,1,False,O,<NA>,O,0,[-1.07308812e-01 -3.37210178e-01 1.22697961e+0...
...,...,...,...,...,...,...,...,...,...,...,...
684,684,"[1621, 1622): ','",117,0,1,False,O,<NA>,O,0,[-1.28065705e-01 -2.32442445e-03 6.78131640e-0...
685,685,"[1623, 1625): 'in'",1107,0,1,False,O,<NA>,O,0,[ 0.30534053 -0.526257 0.8281703 -0.2741491...
686,686,"[1626, 1633): 'Karachi'",16237,0,1,False,B,LOC,B-LOC,1,[-0.04873929 -0.33797333 -0.05835138 0.7557763...
687,687,"[1633, 1634): '.'",119,0,1,False,O,<NA>,O,0,[-5.28932782e-03 -2.97430724e-01 7.16174304e-0...


In [33]:
# Run the entire corpus through our processing pipeline.
bert_toks_by_fold = {}
for fold_name in corpus_raw.keys():
    print(f"Processing fold '{fold_name}'...")
    raw = corpus_raw[fold_name]
    bert_toks_by_fold[fold_name] = tp.jupyter.run_with_progress_bar(
        len(raw), lambda i: tp.io.bert.conll_to_bert(raw[i], tokenizer, 
                                                     bert, token_class_dtype))
bert_toks_by_fold["dev"][20]

Processing fold 'train'...


IntProgress(value=0, description='Starting...', layout=Layout(width='100%'), max=946, style=ProgressStyle(desc…

Processing fold 'dev'...


IntProgress(value=0, description='Starting...', layout=Layout(width='100%'), max=216, style=ProgressStyle(desc…

Processing fold 'test'...


IntProgress(value=0, description='Starting...', layout=Layout(width='100%'), max=231, style=ProgressStyle(desc…

,token_id,span,input_id,token_type_id,attention_mask,special_tokens_mask,ent_iob,ent_type,token_class,token_class_id,embedding
0,0,"[0, 0): ''",101,0,1,True,O,<NA>,O,0,[-0.17669654 -0.39899608 0.9088872 0.2128861...
1,1,"[0, 1): '-'",118,0,1,False,O,<NA>,O,0,[-3.85537773e-01 -5.02327383e-01 1.17323220e+0...
2,2,"[1, 2): 'D'",141,0,1,False,O,<NA>,O,0,[-1.17190674e-01 -1.27011672e-01 1.38969100e+0...
3,3,"[2, 4): 'OC'",9244,0,1,False,O,<NA>,O,0,[-3.90257567e-01 -2.50433564e-01 1.07450700e+0...
4,4,"[4, 6): 'ST'",9272,0,1,False,O,<NA>,O,0,[-2.77328491e-01 -2.61601597e-01 1.07876039e+0...
...,...,...,...,...,...,...,...,...,...,...,...
2154,2154,"[5704, 5705): ')'",114,0,1,False,O,<NA>,O,0,[ 0.01539315 -0.04065072 1.0011852 0.0412664...
2155,2155,"[5706, 5708): '39'",3614,0,1,False,O,<NA>,O,0,[ 7.50392899e-02 1.44007439e-02 1.04323184e+0...
2156,2156,"[5708, 5709): '.'",119,0,1,False,O,<NA>,O,0,[-0.08579681 0.05905596 1.1146404 0.1048710...
2157,2157,"[5709, 5711): '93'",5429,0,1,False,O,<NA>,O,0,[ 1.13780675e-02 -2.63872057e-01 8.81803274e-0...


# Collate the data structures we've generated so far

In [34]:
# Create a single dataframe with the entire corpus's embeddings.
corpus_df = tp.io.conll.combine_folds(bert_toks_by_fold)
corpus_df

,fold,doc_num,token_id,span,input_id,token_type_id,attention_mask,special_tokens_mask,ent_iob,ent_type,token_class,token_class_id,embedding
0,train,0,0,"[0, 0): ''",101,0,1,True,O,<NA>,O,0,[-9.85053703e-02 -4.05019075e-01 7.42887914e-0...
1,train,0,1,"[0, 1): '-'",118,0,1,False,O,<NA>,O,0,[-5.70215769e-02 -4.81121272e-01 9.89868402e-0...
2,train,0,2,"[1, 2): 'D'",141,0,1,False,O,<NA>,O,0,[-4.82420959e-02 -2.53300011e-01 1.16719234e+0...
3,train,0,3,"[2, 4): 'OC'",9244,0,1,False,O,<NA>,O,0,[-2.66830117e-01 -3.10087562e-01 1.00747287e+0...
4,train,0,4,"[4, 6): 'ST'",9272,0,1,False,O,<NA>,O,0,[-0.22296937 -0.2130852 0.93310183 -0.2613601...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
416536,test,230,314,"[1386, 1393): 'brother'",1711,0,1,False,O,<NA>,O,0,[-2.81729121e-02 -8.06238949e-02 9.80488896e-0...
416537,test,230,315,"[1393, 1394): ','",117,0,1,False,O,<NA>,O,0,[ 1.18173949e-01 -7.00848699e-02 8.65484953e-0...
416538,test,230,316,"[1395, 1400): 'Bobby'",5545,0,1,False,B,PER,B-PER,4,[-3.56894344e-01 3.14004630e-01 1.57385385e+0...
416539,test,230,317,"[1400, 1401): '.'",119,0,1,False,O,<NA>,O,0,[-1.89571261e-01 -2.45812088e-01 6.62573814e-0...


# Checkpoint
With the TensorArray from Text Extension for Pandas, the computed embeddings can be persisted as a tensor along with the rest of the DataFrame using standard Pandas input/output methods. Since this is a costly operation and the embeddings are deterministic, it can save lots of time to checkpoint the data here and save the results to disk. This will allow us to continue working with model training without needing to re-compute the BERT embeddings again.
 
 ## Save DataFrame with Embeddings Tensor

In [35]:
# Write the tokenized corpus with embeddings to a Feather file.
# We can't currently serialize span columns that cover multiple documents (see issue 73),
# so drop span columns from the contents we write to the Feather file.
cols_to_drop = [c for c in corpus_df.columns if "span" in c]
corpus_df.drop(columns=cols_to_drop).to_feather("outputs/corpus.feather")

## Load DataFrame with Previously Computed Embeddings

In [36]:
# Read the serialized embeddings back in so that you can rerun the model 
# training parts of this notebook (the cells from here onward) without 
# regenerating the embeddings.
corpus_df = pd.read_feather("outputs/corpus.feather")
corpus_df

,fold,doc_num,token_id,input_id,token_type_id,attention_mask,special_tokens_mask,ent_iob,ent_type,token_class,token_class_id,embedding
0,train,0,0,101,0,1,True,O,<NA>,O,0,[-9.85053703e-02 -4.05019075e-01 7.42887914e-0...
1,train,0,1,118,0,1,False,O,<NA>,O,0,[-5.70215769e-02 -4.81121272e-01 9.89868402e-0...
2,train,0,2,141,0,1,False,O,<NA>,O,0,[-4.82420959e-02 -2.53300011e-01 1.16719234e+0...
3,train,0,3,9244,0,1,False,O,<NA>,O,0,[-2.66830117e-01 -3.10087562e-01 1.00747287e+0...
4,train,0,4,9272,0,1,False,O,<NA>,O,0,[-0.22296937 -0.2130852 0.93310183 -0.2613601...
...,...,...,...,...,...,...,...,...,...,...,...,...
416536,test,230,314,1711,0,1,False,O,<NA>,O,0,[-2.81729121e-02 -8.06238949e-02 9.80488896e-0...
416537,test,230,315,117,0,1,False,O,<NA>,O,0,[ 1.18173949e-01 -7.00848699e-02 8.65484953e-0...
416538,test,230,316,5545,0,1,False,B,PER,B-PER,4,[-3.56894344e-01 3.14004630e-01 1.57385385e+0...
416539,test,230,317,119,0,1,False,O,<NA>,O,0,[-1.89571261e-01 -2.45812088e-01 6.62573814e-0...


# Training a model on the BERT embeddings

Now we will use the loaded BERT embeddings to train a multinomial model to predict the token class from the embeddings tensor.

In [37]:
# Extract the training set DataFrame
train_df = corpus_df[corpus_df["fold"] == "train"]
train_df

,fold,doc_num,token_id,input_id,token_type_id,attention_mask,special_tokens_mask,ent_iob,ent_type,token_class,token_class_id,embedding
0,train,0,0,101,0,1,True,O,<NA>,O,0,[-9.85053703e-02 -4.05019075e-01 7.42887914e-0...
1,train,0,1,118,0,1,False,O,<NA>,O,0,[-5.70215769e-02 -4.81121272e-01 9.89868402e-0...
2,train,0,2,141,0,1,False,O,<NA>,O,0,[-4.82420959e-02 -2.53300011e-01 1.16719234e+0...
3,train,0,3,9244,0,1,False,O,<NA>,O,0,[-2.66830117e-01 -3.10087562e-01 1.00747287e+0...
4,train,0,4,9272,0,1,False,O,<NA>,O,0,[-0.22296937 -0.2130852 0.93310183 -0.2613601...
...,...,...,...,...,...,...,...,...,...,...,...,...
281104,train,945,53,17057,0,1,False,B,ORG,B-ORG,3,[ 7.55636692e-01 -9.18912172e-01 -1.40303954e-0...
281105,train,945,54,122,0,1,False,O,<NA>,O,0,[-1.15284957e-01 -4.44919914e-01 4.71556604e-0...
281106,train,945,55,4617,0,1,False,B,ORG,B-ORG,3,[ 4.56021696e-01 -8.97085190e-01 6.78616092e-0...
281107,train,945,56,123,0,1,False,O,<NA>,O,0,[-1.97137520e-01 -5.42719424e-01 2.94020921e-0...


In [40]:
# Train a multinomial logistic regression model on the training set.
MULTI_CLASS = "multinomial"
    
# How many iterations to run the BGFS optimizer when fitting logistic
# regression models. 100 ==> Fast; 10000 ==> Full convergence
LBGFS_ITERATIONS = 10000

base_pipeline = sklearn.pipeline.Pipeline([
    # Standard scaler. This only makes a difference for certain classes
    # of embeddings.
    #("scaler", sklearn.preprocessing.StandardScaler()),
    ("mlogreg", sklearn.linear_model.LogisticRegression(
        multi_class=MULTI_CLASS,
        verbose=10,
        max_iter=LBGFS_ITERATIONS
    ))
])

X_train = train_df["embedding"].values
Y_train = train_df["token_class_id"]
base_model = base_pipeline.fit(X_train, Y_train)
base_model

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 17.8min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 17.8min finished


Pipeline(steps=[('mlogreg',
                 LogisticRegression(max_iter=10000, multi_class='multinomial',
                                    verbose=10))])

## Make Predictions on Token Class from BERT Embeddings

Using our model, we can now predict the token class from the test set using the computed embeddings.

In [ ]:
# TODO - nicer display of tensor float values?
#import sys
#np.set_printoptions(precision=4, suppress=True, floatmode="fixed", linewidth=100, threshold=sys.maxsize)

In [42]:
# Define a function that will let us make predictions on a fold of the corpus
def predict_on_df(df: pd.DataFrame, id_to_class: Dict[int, str], predictor):
    """
    Run a trained model on a DataFrame of tokens with embeddings.

    :param df: DataFrame of tokens for a document, containing a TokenSpan column
     called "embedding" for each token.
    :param id_to_class: Mapping from class ID to class name, as returned by
     :func:`text_extensions_for_pandas.make_iob_tag_categories`
    :param predictor: Python object with a `predict_proba` method that accepts
     a numpy array of embeddings.
    :returns: A copy of `df`, with the following additional columns:
     `predicted_id`, `predicted_class`, `predicted_iob`, `predicted_type`
     and `predicted_class_pr`.
    """
    result_df = df.copy()
    class_pr = tp.TensorArray(predictor.predict_proba(result_df["embedding"]))
    result_df["predicted_id"] = np.argmax(class_pr, axis=1)
    result_df["predicted_class"] = [id_to_class[i]
                                    for i in result_df["predicted_id"].values]
    iobs, types = tp.io.conll.decode_class_labels(result_df["predicted_class"].values)
    result_df["predicted_iob"] = iobs
    result_df["predicted_type"] = types
    result_df["predicted_class_pr"] = class_pr
    return result_df

In [44]:
# Make predictions on the test set
test_results_df = predict_on_df(corpus_df[corpus_df["fold"] == "test"], int_to_label, base_model)
test_results_df.head()

,fold,doc_num,token_id,input_id,token_type_id,attention_mask,special_tokens_mask,ent_iob,ent_type,token_class,token_class_id,embedding,predicted_id,predicted_class,predicted_iob,predicted_type,predicted_class_pr
351001,test,0,0,101,0,1,True,O,<NA>,O,0,[-1.96265429e-01 -4.50937688e-01 6.77535713e-0...,0,O,O,None,[9.99636348e-01 3.38372009e-06 8.82776168e-05 8...
351002,test,0,1,118,0,1,False,O,<NA>,O,0,[-3.18721086e-01 -5.07478833e-01 1.04645443e+0...,0,O,O,None,[9.99869294e-01 9.26613782e-08 2.55152780e-06 2...
351003,test,0,2,141,0,1,False,O,<NA>,O,0,[-0.08053827 -0.24774824 1.3562553 0.4017882...,0,O,O,None,[9.99850718e-01 1.05468701e-04 1.56706498e-05 4...
351004,test,0,3,9244,0,1,False,O,<NA>,O,0,[-6.87857866e-01 -3.02902311e-01 8.84272158e-0...,0,O,O,None,[9.99681969e-01 6.38913727e-11 4.56670269e-07 4...
351005,test,0,4,9272,0,1,False,O,<NA>,O,0,[-2.96323150e-01 -2.33131766e-01 9.39883947e-0...,0,O,O,None,[9.99997132e-01 3.27647798e-13 3.09048212e-09 1...


In [45]:
# Take a slice to show a region with more entities
test_results_df.iloc[40:50]

,fold,doc_num,token_id,input_id,token_type_id,attention_mask,special_tokens_mask,ent_iob,ent_type,token_class,token_class_id,embedding,predicted_id,predicted_class,predicted_iob,predicted_type,predicted_class_pr
351041,test,0,40,3309,0,1,False,I,PER,I-PER,8,[-2.10291892e-01 -8.53567719e-01 2.75510421e-0...,6,I-MISC,I,MISC,[1.98323229e-07 1.53130728e-10 2.78405342e-07 2...
351042,test,0,41,1306,0,1,False,I,PER,I-PER,8,[-2.32054338e-01 -9.29077148e-01 3.88911217e-0...,6,I-MISC,I,MISC,[3.10633651e-05 5.22947228e-06 6.52603219e-11 5...
351043,test,0,42,2001,0,1,False,I,PER,I-PER,8,[ 0.36844087 -0.6809115 -0.10591034 -0.3448123...,8,I-PER,I,PER,[2.00370294e-03 1.00450102e-01 6.38725156e-06 1...
351044,test,0,43,1181,0,1,False,I,PER,I-PER,8,[-3.01310837e-01 -6.54601932e-01 -1.72691718e-0...,8,I-PER,I,PER,[1.12122699e-04 3.13711716e-06 1.53931420e-07 4...
351045,test,0,44,2293,0,1,False,I,PER,I-PER,8,[-1.61161020e-01 -6.98910296e-01 2.34246060e-0...,5,I-LOC,I,LOC,[7.02232647e-03 3.18677296e-03 7.76558124e-08 6...
351046,test,0,45,18589,0,1,False,B,LOC,B-LOC,1,[-5.85671440e-02 -7.95588315e-01 3.36059839e-0...,1,B-LOC,B,LOC,[6.36387711e-03 9.02478479e-01 2.26270005e-06 4...
351047,test,0,46,118,0,1,False,I,LOC,I-LOC,5,[ 2.03760266e-01 -7.37309694e-01 -8.88528451e-0...,5,I-LOC,I,LOC,[2.00492628e-01 5.12452116e-05 1.72780996e-07 8...
351048,test,0,47,19016,0,1,False,I,LOC,I-LOC,5,[-1.03412405e-01 -3.36818755e-01 1.73846602e-0...,1,B-LOC,B,LOC,[3.35755616e-02 5.33375311e-01 1.00517707e-07 9...
351049,test,0,48,2249,0,1,False,I,LOC,I-LOC,5,[-4.05427039e-01 -6.51651978e-01 2.46960565e-0...,5,I-LOC,I,LOC,[3.03682213e-04 1.33970366e-04 1.83506170e-08 1...
351050,test,0,49,117,0,1,False,O,<NA>,O,0,[-1.68292239e-01 -6.47586107e-01 8.14902484e-0...,0,O,O,None,[9.99999663e-01 3.75752227e-08 1.14784193e-09 1...


### Convert IOB2 token class predictions to named entities

**TODO** explain why

In [47]:
def align_model_outputs_to_tokens(model_results: pd.DataFrame,
                                  tokens_by_doc: Dict[str, List[pd.DataFrame]]) \
    -> Dict[Tuple[str, int], pd.DataFrame]:
    """
    Join the results of running a model on an entire corpus back with multiple
    DataFrames of the token features for the individual documents.

    :param model_results: DataFrame containing results of prediction over a
     collection of documents. Must have the fields:
     * `fold`: What fold of the original collection each document came from
     * `doc_num`: Index of the document within the fold
     * `token_id`: Token offset of the token
     * `predicted_iob`/`predicted_type`: Model outputs
     Usually this DataFrame is the result of running :func:`predict_on_df()`
    :param tokens_by_doc: One DataFrame of tokens and labels per document,
     indexed by fold and document number (which must align with the values
     in the "fold" and "doc_num" columns of `model_results`).
     These DataFrames must contain columns `ent_iob` and `ent_type` that
     correspond to the `predicted_iob` and `predicted_type` values in
     `model_results`

    :returns: A dictionary that maps (collection, offset into collection)
     to DataFrame of results for that document
    """
    all_pairs = (
        model_results[["fold", "doc_num"]]
            .drop_duplicates()
            .to_records(index=False)
    )
    indexed_df = (
        model_results
            .set_index(["fold", "doc_num", "token_id"], verify_integrity=True)
            .sort_index()
    )
    results = {}  # Type: Dict[Tuple[str, int], pd.DataFrame]
    for collection, doc_num in all_pairs:
        doc_slice = indexed_df.loc[collection, doc_num].reset_index()
        doc_toks = tokens_by_doc[collection][doc_num][
            ["token_id", "span", "ent_iob", "ent_type"]
        ].rename(columns={"id": "token_id"})
        result_df = doc_toks.copy().merge(
            doc_slice[["token_id", "predicted_iob", "predicted_type", "predicted_class_pr"]])
        results[(collection, doc_num)] = result_df
    return results

In [48]:
# Split model outputs for an entire fold back into documents and add
# token information.
test_results_by_doc = align_model_outputs_to_tokens(test_results_df,
                                                    bert_toks_by_fold)
test_results_by_doc[("test", 0)].iloc[40:60]

,token_id,span,ent_iob,ent_type,predicted_iob,predicted_type,predicted_class_pr
40,40,"[68, 70): 'di'",I,PER,I,MISC,[1.98323229e-07 1.53130728e-10 2.78405342e-07 2...
41,41,"[70, 71): 'm'",I,PER,I,MISC,[3.10633651e-05 5.22947228e-06 6.52603219e-11 5...
42,42,"[72, 74): 'La'",I,PER,I,PER,[2.00370294e-03 1.00450102e-01 6.38725156e-06 1...
43,43,"[74, 75): 'd'",I,PER,I,PER,[1.12122699e-04 3.13711716e-06 1.53931420e-07 4...
44,44,"[75, 77): 'ki'",I,PER,I,LOC,[7.02232647e-03 3.18677296e-03 7.76558124e-08 6...
45,45,"[78, 80): 'AL'",B,LOC,B,LOC,[6.36387711e-03 9.02478479e-01 2.26270005e-06 4...
46,46,"[80, 81): '-'",I,LOC,I,LOC,[2.00492628e-01 5.12452116e-05 1.72780996e-07 8...
47,47,"[81, 83): 'AI'",I,LOC,B,LOC,[3.35755616e-02 5.33375311e-01 1.00517707e-07 9...
48,48,"[83, 84): 'N'",I,LOC,I,LOC,[3.03682213e-04 1.33970366e-04 1.83506170e-08 1...
49,49,"[84, 85): ','",O,<NA>,O,None,[9.99999663e-01 3.75752227e-08 1.14784193e-09 1...


In [52]:
# Convert IOB2-format output (and gold standard tags) to spans.
test_actual_spans = {k: tp.io.conll.iob_to_spans(v) for k, v in test_results_by_doc.items()}
test_model_spans = {k:
        tp.io.conll.iob_to_spans(v, iob_col_name = "predicted_iob",
                                 entity_type_col_name = "predicted_type")
            .rename(columns={"predicted_type": "ent_type"})
        for k, v in test_results_by_doc.items()}

test_model_spans[("test", 0)].head()

,span,ent_type
0,"[19, 24): 'JAPAN'",PER
1,"[29, 34): 'LUCKY'",LOC
2,"[40, 45): 'CHINA'",ORG
3,"[78, 81): 'AL-'",LOC
4,"[81, 84): 'AIN'",LOC


In [53]:
# Every once in a while, the model will split a token in the original data
# set into two entities. For example, look at document 202 of the test
# set:
doc_key = ("test", 202)
test_model_spans[doc_key].head(10)

,span,ent_type
0,"[11, 22): 'RUGBY UNION'",ORG
1,"[24, 31): 'BRITISH'",MISC
2,"[41, 47): 'LONDON'",LOC
3,"[70, 77): 'British'",MISC
4,"[111, 125): 'Pilkington Cup'",MISC
5,"[139, 146): 'Reading'",ORG
6,"[150, 151): 'W'",ORG
7,"[151, 156): 'idnes'",ORG
8,"[159, 166): 'English'",MISC
9,"[180, 184): 'Bath'",ORG


In [56]:
# Notice `[150, 151): 'W'` and `[151, 156): 'idnes'`. These outputs are part
# of the same original token.
# We can use spanner algebra to fix up these outputs.
spans_df = test_model_spans[doc_key]
toks_df = test_raw[202]

# First, find which tokens the spans overlap with:
overlaps_df = (
    tp.spanner.overlap_join(spans_df["span"], toks_df["span"],
                            "span", "corpus_token")
        .merge(spans_df)
)
overlaps_df.head(10)

,span,corpus_token,ent_type
0,"[11, 22): 'RUGBY UNION'","[11, 16): 'RUGBY'",ORG
1,"[11, 22): 'RUGBY UNION'","[17, 22): 'UNION'",ORG
2,"[24, 31): 'BRITISH'","[24, 31): 'BRITISH'",MISC
3,"[41, 47): 'LONDON'","[41, 47): 'LONDON'",LOC
4,"[70, 77): 'British'","[70, 77): 'British'",MISC
5,"[111, 125): 'Pilkington Cup'","[111, 121): 'Pilkington'",MISC
6,"[111, 125): 'Pilkington Cup'","[122, 125): 'Cup'",MISC
7,"[139, 146): 'Reading'","[139, 146): 'Reading'",ORG
8,"[150, 151): 'W'","[150, 156): 'Widnes'",ORG
9,"[151, 156): 'idnes'","[150, 156): 'Widnes'",ORG


In [58]:
# Next, compute the minimum span that covers all the corpus tokens
# that overlap with each entity span.
agg_df = (
    overlaps_df
    .groupby("span")
    .aggregate({"corpus_token": "sum", "ent_type": "first"})
    .reset_index()
)
agg_df.head(10)

,span,corpus_token,ent_type
0,"[11, 22): 'RUGBY UNION'","[11, 22): 'RUGBY UNION'",ORG
1,"[24, 31): 'BRITISH'","[24, 31): 'BRITISH'",MISC
2,"[41, 47): 'LONDON'","[41, 47): 'LONDON'",LOC
3,"[70, 77): 'British'","[70, 77): 'British'",MISC
4,"[111, 125): 'Pilkington Cup'","[111, 125): 'Pilkington Cup'",MISC
5,"[139, 146): 'Reading'","[139, 146): 'Reading'",ORG
6,"[150, 151): 'W'","[150, 156): 'Widnes'",ORG
7,"[151, 156): 'idnes'","[150, 156): 'Widnes'",ORG
8,"[159, 166): 'English'","[159, 166): 'English'",MISC
9,"[180, 184): 'Bath'","[180, 184): 'Bath'",ORG


In [61]:
# Finally, take unique values and covert character-based spans to spans
# in the corpus tokenization (since the new offsets might not match a
# BERT tokenizer token boundary)
cons_df = (
    tp.spanner.consolidate(agg_df, "corpus_token")[["corpus_token", "ent_type"]]
        .rename(columns={"corpus_token": "span"})
)
cons_df["span"] = tp.TokenSpanArray.align_to_tokens(toks_df["span"],
                                                    cons_df["span"])
cons_df.head(10)

,span,ent_type
0,"[11, 22): 'RUGBY UNION'",ORG
1,"[24, 31): 'BRITISH'",MISC
2,"[41, 47): 'LONDON'",LOC
3,"[70, 77): 'British'",MISC
4,"[111, 125): 'Pilkington Cup'",MISC
5,"[139, 146): 'Reading'",ORG
6,"[150, 156): 'Widnes'",ORG
8,"[159, 166): 'English'",MISC
9,"[180, 184): 'Bath'",ORG
10,"[188, 198): 'Harlequins'",ORG


In [64]:
# text_extensions_for_pandas contains a single function that repeats the actions of the 
# previous 3 cells.
tp.io.bert.align_bert_tokens_to_corpus_tokens(test_model_spans[doc_key], test_raw[202]).head(10)

,span,ent_type,token_span
0,"[11, 22): 'RUGBY UNION'",ORG,"[11, 22): 'RUGBY UNION'"
1,"[24, 31): 'BRITISH'",MISC,"[24, 31): 'BRITISH'"
2,"[41, 47): 'LONDON'",LOC,"[41, 47): 'LONDON'"
3,"[70, 77): 'British'",MISC,"[70, 77): 'British'"
4,"[111, 125): 'Pilkington Cup'",MISC,"[111, 125): 'Pilkington Cup'"
5,"[139, 146): 'Reading'",ORG,"[139, 146): 'Reading'"
6,"[150, 156): 'Widnes'",ORG,"[150, 156): 'Widnes'"
8,"[159, 166): 'English'",MISC,"[159, 166): 'English'"
9,"[180, 184): 'Bath'",ORG,"[180, 184): 'Bath'"
10,"[188, 198): 'Harlequins'",ORG,"[188, 198): 'Harlequins'"


In [67]:
# Run all of our dataframes through `align_bert_tokens_to_corpus_tokens`
keys = list(test_model_spans.keys())
new_values = tp.jupyter.run_with_progress_bar(
    len(keys), 
    lambda i: tp.io.bert.align_bert_tokens_to_corpus_tokens(test_model_spans[keys[i]], test_raw[keys[i][1]]))
test_model_spans = {k: v for k, v in zip(keys, new_values)}
test_model_spans[doc_key].head(10)

IntProgress(value=0, description='Starting...', layout=Layout(width='100%'), max=231, style=ProgressStyle(desc…

,span,ent_type,token_span
0,"[11, 22): 'RUGBY UNION'",ORG,"[11, 22): 'RUGBY UNION'"
1,"[24, 31): 'BRITISH'",MISC,"[24, 31): 'BRITISH'"
2,"[41, 47): 'LONDON'",LOC,"[41, 47): 'LONDON'"
3,"[70, 77): 'British'",MISC,"[70, 77): 'British'"
4,"[111, 125): 'Pilkington Cup'",MISC,"[111, 125): 'Pilkington Cup'"
5,"[139, 146): 'Reading'",ORG,"[139, 146): 'Reading'"
6,"[150, 156): 'Widnes'",ORG,"[150, 156): 'Widnes'"
8,"[159, 166): 'English'",MISC,"[159, 166): 'English'"
9,"[180, 184): 'Bath'",ORG,"[180, 184): 'Bath'"
10,"[188, 198): 'Harlequins'",ORG,"[188, 198): 'Harlequins'"


## Compute Precision/Recall

In [69]:
# Same per-document statistics calculation code as in CoNLL_2.ipynb
test_stats_by_doc = tp.io.conll.compute_accuracy_by_document(test_actual_spans, test_model_spans)
test_stats_by_doc

,fold,doc_num,num_true_positives,num_extracted,num_entities,precision,recall,F1
0,test,0,42,46,45,0.913043,0.933333,0.923077
1,test,1,41,44,44,0.931818,0.931818,0.931818
2,test,2,52,54,54,0.962963,0.962963,0.962963
3,test,3,42,44,44,0.954545,0.954545,0.954545
4,test,4,18,19,19,0.947368,0.947368,0.947368
...,...,...,...,...,...,...,...,...
226,test,226,7,7,7,1.000000,1.000000,1.000000
227,test,227,18,20,21,0.900000,0.857143,0.878049
228,test,228,23,28,27,0.821429,0.851852,0.836364
229,test,229,26,27,27,0.962963,0.962963,0.962963


In [70]:
# Collection-wide precision and recall can be computed by aggregating
# our dataframe.
tp.io.conll.compute_global_accuracy(test_stats_by_doc)

{'num_true_positives': 4924,
 'num_entities': 5648,
 'num_extracted': 5600,
 'precision': 0.8792857142857143,
 'recall': 0.8718130311614731,
 'F1': 0.8755334281650072}

## [WIP] Further Analysis

In [77]:
# Find the predictions that had the lowest probability to see what the model did not predict strongly
# We can easily mix numpy functions on the TensorArray with standard Pandas on the entire DataFrame
# Here we find the max probability for each prediction, then sort the DataFrame by that value ascending
test_results_df["class_max_pr"] = np.max(test_results_df["predicted_class_pr"].array, axis=1)
pr_asc_df = test_results_df.sort_values(by="class_max_pr")

# Select relevant columns and show lowest 20 predictions
pr_asc_df[["doc_num", "token_id", "token_class", "predicted_class", "class_max_pr", "predicted_class_pr"]].head(20)

,doc_num,token_id,token_class,predicted_class,class_max_pr,predicted_class_pr
414699,222,371,O,B-MISC,0.238123,[0.08202944 0.00349575 0.23812346 0.14204107 0....
390307,133,8,B-LOC,B-MISC,0.291589,[2.62719988e-01 2.38731092e-01 2.91588782e-01 3...
367670,54,49,I-ORG,I-ORG,0.294557,[1.76412153e-01 3.86455931e-07 4.47598135e-07 7...
363788,42,28,O,O,0.309699,[3.09698791e-01 1.89220263e-05 4.49656243e-05 5...
411548,213,14,I-MISC,I-LOC,0.317874,[2.89094277e-01 3.86128605e-05 1.78886350e-04 3...
385606,114,10,I-LOC,B-MISC,0.327414,[2.28951121e-01 2.04707195e-06 3.27413638e-01 1...
358565,23,387,B-ORG,B-ORG,0.327566,[2.80056412e-01 1.09979216e-01 9.60765376e-02 3...
358426,23,248,I-ORG,I-MISC,0.347311,[1.80356203e-02 1.90402981e-03 3.16516185e-02 2...
364233,44,20,I-ORG,I-PER,0.348541,[1.22118924e-03 3.17763625e-05 1.27397341e-05 4...
386205,116,116,O,I-PER,0.350508,[3.24629125e-01 2.88704564e-05 1.68123889e-05 8...


In [78]:
# Let's take a closer look at the most predictions with the lowest probability and see if there is a document that had the most difficulties
lowest_pr_df = pr_asc_df[["doc_num", "token_id", "input_id", "predicted_class", "class_max_pr"]].head(100)
lowest_pr_df["doc_num"].value_counts().head(5)

23     11
27     10
222     6
220     5
133     3
Name: doc_num, dtype: int64

In [ ]:
# It looks like document 23 of the test set had the most weak predictions, let's take a look at this document and see which spans caused this
# TODO - want to show the span with predictions with this document
all_spans = {
    k: [tp.io.conll.iob_to_spans(df) for df in v] for k, v in corpus_raw.items()
}
fold = "test"
doc_offset = 23
doc_df = all_spans[fold][doc_offset]
doc_df

In [62]:
corpus_test = corpus_df[corpus_df["fold"] == "test"]
doc_23 = corpus_test[corpus_test["doc_num"] == 23]

In [118]:
q = test_results_by_doc[("test"), 23].iloc[387:392]
q

,token_id,span,ent_iob,ent_type,predicted_iob,predicted_type,predicted_class_pr,token_span
387,387,"[911, 913): 'VA'",B,ORG,B,ORG,[2.80056412e-01 1.09979216e-01 9.60765376e-02 3...,"[911, 913): 'VA'"
388,388,"[913, 915): 'NC'",I,ORG,O,None,[5.07208264e-01 7.57895366e-06 9.71331129e-06 6...,"[913, 915): 'NC'"
389,389,"[915, 916): 'O'",I,ORG,O,None,[5.58220550e-01 4.23603022e-06 3.18193166e-05 1...,"[915, 916): 'O'"
390,390,"[916, 917): 'U'",I,ORG,O,None,[7.19035426e-01 7.66025877e-06 6.77641140e-05 3...,"[916, 917): 'U'"
391,391,"[917, 919): 'VE'",I,ORG,I,ORG,[3.10879823e-01 2.03345842e-05 4.74213364e-05 3...,"[917, 919): 'VE'"


In [136]:
pd.DataFrame(q["predicted_class_pr"].array.to_numpy(), columns=int_to_label)

,O,B-LOC,B-MISC,B-ORG,B-PER,I-LOC,I-MISC,I-ORG,I-PER
0,0.280056,0.109979,0.096077,0.327566,0.158920,0.000026,0.000165,0.026778,0.000433
1,0.507208,0.000008,0.000010,0.000006,0.000001,0.000704,0.014455,0.472207,0.005401
2,0.558221,0.000004,0.000032,0.000016,0.000017,0.001999,0.032320,0.212432,0.194960
3,0.719035,0.000008,0.000068,0.000039,0.000016,0.000096,0.021634,0.044089,0.215015
4,0.310880,0.000020,0.000047,0.000342,0.000002,0.002338,0.004121,0.365688,0.316562


In [ ]:
# TODO: trying to get the char span for tokens
lowest_pr_df = lowest_pr_df.merge(test_df, on=["doc_num", "token_id"])
lowest_pr_df = lowest_pr_df[["doc_num", "token_id", "token_class", "predicted_class", "class_max_pr"]]
lowest_pr_df

In [71]:
example_df = corpus_raw["test"][23]
spans_df = tp.io.conll.iob_to_spans(example_df)
bert_toks_df = tp.io.bert.make_bert_tokens(example_df["span"].values[0].target_text, tokenizer)
bert_token_spans = tp.TokenSpanArray.align_to_tokens(bert_toks_df["span"],
                                                     spans_df["span"])
bert_toks_df[["ent_iob", "ent_type"]] = tp.io.conll.spans_to_iob(bert_token_spans, 
                                                                 spans_df["ent_type"])
bert_toks_df

,token_id,span,input_id,token_type_id,attention_mask,special_tokens_mask,ent_iob,ent_type
0,0,"[0, 0): ''",101,0,1,True,O,<NA>
1,1,"[0, 1): '-'",118,0,1,False,O,<NA>
2,2,"[1, 2): 'D'",141,0,1,False,O,<NA>
3,3,"[2, 4): 'OC'",9244,0,1,False,O,<NA>
4,4,"[4, 6): 'ST'",9272,0,1,False,O,<NA>
...,...,...,...,...,...,...,...,...
540,540,"[1228, 1230): 'DM'",20002,0,1,False,I,LOC
541,541,"[1230, 1232): 'ON'",11414,0,1,False,I,LOC
542,542,"[1232, 1234): 'TO'",18082,0,1,False,I,LOC
543,543,"[1234, 1235): 'N'",2249,0,1,False,I,LOC


In [74]:
test_results_by_doc[("test", 23)].head(50)

,token_id,span,ent_iob,ent_type,predicted_iob,predicted_type,predicted_class_pr,token_span
0,0,"[0, 0): ''",O,<NA>,O,None,[9.98777674e-01 2.63167310e-05 1.86662206e-04 2...,"[0, 0): ''"
1,1,"[0, 1): '-'",O,<NA>,O,None,[9.99608524e-01 1.01706951e-06 9.78275812e-06 4...,"[0, 1): '-'"
2,2,"[1, 2): 'D'",O,<NA>,O,None,[9.99793594e-01 9.76208608e-05 2.39952739e-05 3...,"[1, 2): 'D'"
3,3,"[2, 4): 'OC'",O,<NA>,O,None,[9.93425837e-01 6.73497081e-08 1.16274335e-05 1...,"[2, 4): 'OC'"
4,4,"[4, 6): 'ST'",O,<NA>,O,None,[9.99922319e-01 1.13388610e-10 9.30916090e-08 1...,"[4, 6): 'ST'"
5,5,"[6, 8): 'AR'",O,<NA>,O,None,[9.99399761e-01 9.48746013e-07 5.95919740e-05 4...,"[6, 8): 'AR'"
6,6,"[8, 9): 'T'",O,<NA>,O,None,[9.96468727e-01 3.85340552e-08 7.21067291e-07 1...,"[8, 9): 'T'"
7,7,"[9, 10): '-'",O,<NA>,O,None,[9.98293417e-01 5.46407023e-06 2.36536592e-05 2...,"[9, 10): '-'"
8,8,"[11, 14): 'NHL'",B,ORG,B,ORG,[2.24790607e-05 2.45303321e-03 6.24772793e-04 9...,"[11, 14): 'NHL'"
9,9,"[15, 16): 'I'",O,<NA>,O,None,[9.92729478e-01 7.66368904e-06 1.53463530e-04 8...,"[15, 16): 'I'"


In [ ]:
test_model_spans[("test", 23)]

In [80]:
bert_toks_by_fold["test"][23].iloc[380:400]

,token_id,span,input_id,token_type_id,attention_mask,special_tokens_mask,ent_iob,ent_type,token_class,token_class_id,embedding
380,380,"[893, 894): 'O'",2346,0,1,False,I,ORG,I-ORG,7,[-9.56230640e-01 -8.52525651e-01 -2.55697429e-0...
381,381,"[895, 897): '17'",1542,0,1,False,O,<NA>,O,0,[-7.53681421e-01 -5.34573615e-01 2.97853976e-0...
382,382,"[898, 899): '6'",127,0,1,False,O,<NA>,O,0,[-5.08122385e-01 -3.48633051e-01 7.61998892e-0...
383,383,"[900, 901): '4'",125,0,1,False,O,<NA>,O,0,[-5.12067854e-01 -3.19513857e-01 8.18652749e-0...
384,384,"[902, 904): '97'",5311,0,1,False,O,<NA>,O,0,[-4.32366520e-01 -3.36563289e-01 8.48116577e-0...
385,385,"[905, 907): '56'",4376,0,1,False,O,<NA>,O,0,[-4.38618928e-01 -3.16424578e-01 8.72659743e-0...
386,386,"[908, 910): '38'",3383,0,1,False,O,<NA>,O,0,[-0.47988272 -0.3505001 0.8971388 -0.4028021...
387,387,"[911, 913): 'VA'",19497,0,1,False,B,ORG,B-ORG,3,[-5.86880565e-01 -4.14607137e-01 7.35883355e-0...
388,388,"[913, 915): 'NC'",15517,0,1,False,I,ORG,I-ORG,7,[-7.97141552e-01 -4.29197282e-01 4.51419413e-0...
389,389,"[915, 916): 'O'",2346,0,1,False,I,ORG,I-ORG,7,[-7.90252447e-01 -6.32849455e-01 3.05434912e-0...


In [85]:
temp = bert_toks_by_fold["test"][23].iloc[387]

In [86]:
temp

token_id                                                             387
span                                                    [911, 913): 'VA'
input_id                                                           19497
token_type_id                                                          0
attention_mask                                                         1
special_tokens_mask                                                False
ent_iob                                                                B
ent_type                                                             ORG
token_class                                                        B-ORG
token_class_id                                                         3
embedding              [-5.86880565e-01 -4.14607137e-01  7.35883355e-...
Name: 387, dtype: object